# Как научить компьютер читать? 

Это вторая часть большой открытой лекции "Как научить компьютер читать?" В ней мы будем парсить Твиттер. 

### Ссылки: 

* Токены: https://developer.twitter.com/en/apps/13338986
* Ограничения: https://developer.twitter.com/en/docs/basics/rate-limiting
* Документация пакета для парсинга: https://python-twitter.readthedocs.io/en/latest/twitter.html

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import twitter

# всякие разные id и тп для подключения 
api = twitter.Api(
 consumer_key=' ',
 consumer_secret=' ',
 access_token_key=' ',
 access_token_secret=' '
 )

## 1. Сбор в прошлое по запросам 

Попробуем вытащить паручку твиттов по запросу "фантастические твари".

In [ ]:
terms = 'фантастические твари'

tweets = api.GetSearch(term=terms, lang='ru', result_type='recent', count=3, max_id='')
tweets

[Status(ID=1067568152316469249, ScreenName=fapushek, Created=Tue Nov 27 23:57:36 +0000 2018, Text='RT @metashot: 1918 год - издательство заказывает у Ньюта Скамандера книгу "Фантастические твари".\n1918-1927 гг. - Ньют Скамандер проебывает…'),
 Status(ID=1067567933826703360, ScreenName=destrfmfcr, Created=Tue Nov 27 23:56:44 +0000 2018, Text='Чувствую себя неловко из-за того, что мне и зв новые заходят, и фантастические твари. Все такие контркультурные, а я - нет('),
 Status(ID=1067565247433768960, ScreenName=SolutionOfTea, Created=Tue Nov 27 23:46:04 +0000 2018, Text='RT @metashot: 1918 год - издательство заказывает у Ньюта Скамандера книгу "Фантастические твари".\n1918-1927 гг. - Ньют Скамандер проебывает…')]

Каждый твит довольно сложно устроен. Давайте посмотрим какие у него есть методы и объекты. Попутно отберём только ту информацию, которая нам может быть потенциально интересна.

In [ ]:
print(dir(tweets[-1]))

['AsDict', 'AsJsonString', 'NewFromJsonDict', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_json', 'contributors', 'coordinates', 'created_at', 'created_at_in_seconds', 'current_user_retweet', 'favorite_count', 'favorited', 'full_text', 'geo', 'hashtags', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'location', 'media', 'param_defaults', 'place', 'possibly_sensitive', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'retweet_count', 'retweeted', 'retweeted_status', 'scopes', 'source', 'text', 'truncated', 'tweet_mode', 'urls', 'user', 'user_mentions', 'withheld_copyright', 'withheld_in_countries', 'withheld_scope']


In [ ]:
print(tweets[-1].user.screen_name)
print(tweets[-1].created_at)
print(tweets[-1].text)
print(tweets[-1].full_text)
print(tweets[-1].hashtags)
print(tweets[-1].id)
print(tweets[-1].source)
print(tweets[-1].coordinates)
print(tweets[-1].created_at_in_seconds)
print(tweets[-1].geo)
print(tweets[-1].favorite_count)
print(tweets[-1].retweet_count)
print(tweets[-1].retweeted)

SolutionOfTea
Tue Nov 27 23:46:04 +0000 2018
RT @metashot: 1918 год - издательство заказывает у Ньюта Скамандера книгу "Фантастические твари".
1918-1927 гг. - Ньют Скамандер проебывает…
None
[]
1067565247433768960
<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
None
1543362364
None
0
162
False


In [ ]:
tweets[-1].full_text # Почему-то не работает, многие твиты усечены до 140 символов:( 

Завернём всё это добро в удобную функцию, запиливающую таблицу.

In [ ]:
def create_twdf(tweets):
    """
        Создаёт по твиттам таблицу с данными
    """
    
    df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

    df['full_tweet'] = [tweet.full_text for tweet in tweets]
    df['ID'] = [tweet.id for tweet in tweets]
    df['Username'] = [tweet.user.screen_name for tweet in tweets]
    df['Date'] = [tweet.created_at for tweet in tweets]
    df['Hashtags']  = [tweet.hashtags for tweet in tweets]
    df['Retweeted']  = [tweet.retweeted for tweet in tweets]
    df['Source']  = [tweet.source for tweet in tweets]
    df['Likes'] = [tweet.favorite_count for tweet in tweets]
    df['Retweets'] = [tweet.retweet_count for tweet in tweets]
    df['Date_in_seconds'] = [tweet.created_at_in_seconds for tweet in tweets]
    df['Geo'] = [tweet.geo for tweet in tweets]
    df['Coordinates'] = [tweet.coordinates for tweet in tweets]
    return df

df = create_twdf(tweets)
df.head()  

,Tweets,full_tweet,ID,Username,Date,Hashtags,Retweeted,Source,Likes,Retweets,Date_in_seconds,Geo,Coordinates
0,RT @metashot: 1918 год - издательство заказыва...,None,1067568152316469249,fapushek,Tue Nov 27 23:57:36 +0000 2018,[],False,"<a href=""http://twitter.com/download/android"" ...",0,162,1543363056,None,None
1,"Чувствую себя неловко из-за того, что мне и зв...",None,1067567933826703360,destrfmfcr,Tue Nov 27 23:56:44 +0000 2018,[],False,"<a href=""http://twitter.com/download/android"" ...",0,0,1543363004,None,None
2,RT @metashot: 1918 год - издательство заказыва...,None,1067565247433768960,SolutionOfTea,Tue Nov 27 23:46:04 +0000 2018,[],False,"<a href=""http://twitter.com/download/iphone"" r...",0,162,1543362364,None,None


Выводим на промышленные рельсы. Тестим отмотку по id публикации и смотрим на лимиты

In [ ]:
# Способы проходов: 
# подставлять в since_id и max_id ID твтов
# since и untill то же самое, но по датам

terms = 'богемская рапсодия'

tweets = api.GetSearch(term=terms, lang='ru', result_type='mixed', count=2, max_id='')

df = create_twdf(tweets)
df.head()

,Tweets,full_tweet,ID,Username,Date,Hashtags,Retweeted,Source,Likes,Retweets,Date_in_seconds,Geo,Coordinates
0,#ВечернийУргант. Богемская рапсодия 🔥 https://...,None,1066086262724063233,Urgant_Show,Fri Nov 23 21:49:06 +0000 2018,"[{""text"": ""\u0412\u0435\u0447\u0435\u0440\u043...",False,"<a href=""https://studio.twitter.com"" rel=""nofo...",245,36,1543009746,None,None
1,@thnksmom_ Богемская рапсодия,None,1067566814044004353,mrNiputu,Tue Nov 27 23:52:17 +0000 2018,[],False,"<a href=""http://twitter.com/download/android"" ...",0,0,1543362737,None,None


Команда ниже позволяет понять сколько запросов у нас осталось в текущую пятнадцатиминутку.

In [ ]:
api.rate_limit.resources

{'search': {'/search/tweets': {'limit': 180,
   'remaining': 162,
   'reset': 1543363852}}}

Пишем цикл для сбора по запросу в историю.

In [ ]:
dfs = [ ] # сюда пишем новые таблички

In [ ]:
terms = 'баста'  # термин для поиска

In [ ]:
cur_max_id = ''

for i in range(3):
    try:
        tweets = api.GetSearch(term=terms, lang='ru', result_type='mixed', 
                               count=100,
                               max_id=cur_max_id)
        df = create_twdf(tweets)
        dfs.append(df)
        cur_max_id = str(int(df.ID.iloc[-1]) - 1)
        print(cur_max_id)
        print(api.rate_limit.resources, '\n')
    except:
        break

1067488810559303679
{'search': {'/search/tweets': {'limit': 180, 'remaining': 161, 'reset': 1543363852}}} 

1067452019714207743
{'search': {'/search/tweets': {'limit': 180, 'remaining': 160, 'reset': 1543363852}}} 

1067415955893030912
{'search': {'/search/tweets': {'limit': 180, 'remaining': 159, 'reset': 1543363852}}} 



In [ ]:
dff = pd.concat(dfs).reset_index(drop=True)
dff.shape, len(set(dff.Tweets.values))

((300, 13), 141)

In [ ]:
len(dff.Date.unique())

298

In [ ]:
len(set(dff.Tweets))

141

In [ ]:
dff.Date.apply(lambda x: x.split(' ')[2] + ' ' + x.split(' ')[1] + ' ' + x.split(' ')[-1]).unique()

array(['26 Nov 2018', '24 Nov 2018', '27 Nov 2018'], dtype=object)

In [ ]:
index = np.random.randint(0,100)

print(dff.Tweets.values[index])

RT @meduzaproject: В конце концерта «Мою игру» хором пели все — Баста, Нойз, Мирон, Фейс, Андрей Бледный из группы 25/17 и даже Рома Жиган…


In [ ]:
dff.head()

,Tweets,full_tweet,ID,Username,Date,Hashtags,Retweeted,Source,Likes,Retweets,Date_in_seconds,Geo,Coordinates
0,Oxxxymiron / Баста / Noize MC. Мы будем целико...,None,1067067136651288576,tvrain,Mon Nov 26 14:46:45 +0000 2018,"[{""text"": ""\u044f\u0431\u0443\u0434\u0443\u043...",False,"<a href=""https://about.twitter.com/products/tw...",114,38,1543243605,None,None
1,"#ябудупетьсвоюмузыку\nААААА Баста, Нойз и Окси...",None,1067150377324347392,tvrain,Mon Nov 26 20:17:31 +0000 2018,"[{""text"": ""\u044f\u0431\u0443\u0434\u0443\u043...",False,"<a href=""https://about.twitter.com/products/tw...",185,33,1543263451,None,None
2,"Я БУДУ ПЕТЬ СВОЮ МУЗЫКУ: \nБаста, Oxxxymiron, ...",None,1066372820794556417,norimyxxxo,Sat Nov 24 16:47:47 +0000 2018,[],False,"<a href=""http://twitter.com/download/iphone"" r...",19151,4071,1543078067,None,None
3,I liked a @YouTube video https://t.co/278NZEuA...,None,1067567835113754624,ZKapov,Tue Nov 27 23:56:20 +0000 2018,[],False,"<a href=""https://www.google.com/"" rel=""nofollo...",0,0,1543362980,None,None
4,RT @Rhymeslive: Сегодня на концерте в поддержк...,None,1067567184392658944,krisspiro,Tue Nov 27 23:53:45 +0000 2018,[],False,"<a href=""http://twitter.com/download/android"" ...",0,685,1543362825,None,None


In [ ]:
dff.Tweets[-10:]

290    RT @KoleNikita: Чтобы вы понимали!\n\nНа одной...
291    RT @Rhymeslive: Сегодня на концерте в поддержк...
292               @mudakoff Вырасту – не буду, как Баста
293    RT @studio21ru: Баста, Окси и Нойз исполнили «...
294    RT @Aid_Rostov: #ростов \nБаста\n\nhttps://t.c...
295    RT @KoleNikita: Чтобы вы понимали!\n\nНа одной...
296           @rentvchannel Баста, хаски, собаки что ли?
297    RT @meduzaproject: В конце концерта «Мою игру»...
298    RT @meduzaproject: В конце концерта «Мою игру»...
299           #ростов \nБаста\n\nhttps://t.co/DiHSQCzSkV
Name: Tweets, dtype: object

In [ ]:
dff.to_csv('df_basta.csv', sep='\t')

## 2. Сбор в прошлое по аккаунтам

Второй вариант: собирать в прошлое по аккаунтам.

```
GetUserTimeline(user_id=None, 
                screen_name=None, 
                since_id=None, 
                max_id=None, 
                count=None, 
                include_rts=True, 
                trim_user=False, 
                exclude_replies=False)
```

In [ ]:
name = "realDonaldTrump"
dfs = [ ]

In [ ]:
cur_max_id = ''

for i in range(3):
    try:
        tweets = api.GetSearch(term=terms, lang='ru', result_type='mixed', count=100, max_id=cur_max_id)
        df = create_twdf(tweets)
        dfs.append(df)
        cur_max_id = str(int(df.ID.iloc[-1]) - 1)
        print(cur_max_id)
        print(api.rate_limit.resources, '\n')
    except:
        break

1067488810559303679
{'search': {'/search/tweets': {'limit': 180, 'remaining': 158, 'reset': 1543363852}}} 

1067452019714207743
{'search': {'/search/tweets': {'limit': 180, 'remaining': 157, 'reset': 1543363852}}} 

1067415955893030912
{'search': {'/search/tweets': {'limit': 180, 'remaining': 156, 'reset': 1543363852}}} 



## Ссылки да почиташки

* Код для потокового сбора твитов на промышленной основе:  http://adilmoujahid.com/posts/2014/07/twitter-analytics/
* Сентимент-анализ твиттера Трампа: https://dev.to/rodolfoferro/sentiment-analysis-on-trumpss-tweets-using-python-